In [4]:
import numpy as np
import pandas as pd
import plotly.express as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random
import joblib
import pickle
import fnmatch
import os
from sklearn import metrics 

In [54]:
data=pd.read_csv("data.csv")
data_copy=data.copy()
x_train_systemicity, x_test_systemicity, y_train_systemicity, y_test_systemicity=clean_split_data(data_copy)


ensembleModel="RandomClassifierEnsembles2"

train_ensembleModel(x_train_systemicity, x_test_systemicity, y_train_systemicity, y_test_systemicity, ensembleModel)


new_candidate_df=x_test_systemicity.head(5)
lst=predict(new_candidate_df,ensembleModel)


In [7]:
def predict(new_candidate,Ensembles1):
    dir_path = r'{}/'.format(Ensembles1)
#     print(dir_path)
    count = len(fnmatch.filter(os.listdir(dir_path), '*.joblib'))
    arr_pred= np.empty(len(new_candidate), dtype=object)
    i=0
    for rw in range(len(new_candidate)):
        candidate=new_candidate.iloc[[rw]]
        mdl_idx=random.randint(0, (count-1))
        s1="Model"
        s2=".joblib"
        selectedfile = r'{}/{}{}{}'.format(Ensembles1,s1, mdl_idx, s2)
        selected_model = joblib.load(selectedfile)
        arr_pred[i]=(selected_model.predict(candidate)[0])
        i=i+1
        
#         print(selected_model.predict(candidate))
    return(arr_pred)

In [ ]:
###########################################Evaluation Metric##################################################

In [55]:
ensembleModel="RandomClassifierEnsembles2"
test_data=x_test_systemicity.copy()
test_data['Application_status']=y_test_systemicity
print("Coverage of ",ensembleModel," is :", coverage)
print("Arbitrariness of ",ensembleModel," is :", arbitrariness)
coverage, arbitrariness=evaluate2(test_data,ensembleModel)


Coverage of  RandomClassifierEnsembles2  is : 0.6734434561626429
Arbitrariness of  RandomClassifierEnsembles2  is : 1.297469682854636


30.787801778907244


In [ ]:
def evaluate(test_data,Ensembles1):
    test_data_copy=test_data.loc[test_data['Application_status']=="hired"]
#     y_test_eval=test_data['Application_status']
    x_test_eval=test_data_copy.drop(['Application_status'],axis=1)
    all_hired_ln=len(x_test_eval)
    freq_arr = np.zeros(all_hired_ln)
    for i in range(40):
        y_pred_eval=predict(x_test_eval,Ensembles1)
#         print(y_pred_eval)
        indx=0
        for i in y_pred_eval:
#             print(i)
            if i=="hired":
                freq_arr[indx]=freq_arr[indx]+1
            indx=indx+1    
    
    print(freq_arr)
    
############Distribution chart
    df = pd.DataFrame({ 'TP_Frequency': freq_arr })
    df['Candidates'] = range(all_hired_ln)
#     print(df)
    fig = plt.scatter(x=df['Candidates'], y=df['TP_Frequency'])
    fig.show()
    
#######Metric1: Coverage (Indicates fairness of the random classifier ensamble)
    all_TP=len(np.nonzero(freq_arr)[0])
    coverage=all_TP/all_hired_ln
#     print(coverage)
    
    
    
#######Metric2: Arbitrariness (Indicates the arbitrariness of the model and the distribution of true positive)
    TP_freq_mean=np.mean(freq_arr)
    print(TP_freq_mean)
    sum=0
    for j in freq_arr:
        sum=sum+abs(j-TP_freq_mean)
    
#     print(sum)
    arbitrariness=(sum/all_hired_ln)
#     print(all_hired_ln/sum)
        
    return coverage, arbitrariness
        

In [49]:
def evaluate2(test_data,Ensembles2):
    test_data_copy=test_data.loc[test_data['Application_status']=="hired"]
    x_test_eval=test_data_copy.drop(['Application_status'],axis=1)
    all_hired_ln=len(x_test_eval)
    freq_arr = np.zeros(all_hired_ln)
    ######################
    dir_path = r'{}/'.format(Ensembles2)
    count = len(fnmatch.filter(os.listdir(dir_path), '*.joblib'))
    model_indx=0
    for rw in range(count):
        
        s1="Model"
        s2=".joblib"
        selectedfile = r'{}/{}{}{}'.format(Ensembles2,s1, model_indx, s2)
        selected_model = joblib.load(selectedfile)
        y_pred_eval=selected_model.predict(x_test_eval)
        model_indx=model_indx+1
    #####################
    
        indx=0
        for i in y_pred_eval:
            if i=="hired":
                freq_arr[indx]=freq_arr[indx]+1
            indx=indx+1    

    
############Distribution chart
    df = pd.DataFrame({ 'TP_Frequency': freq_arr })
    df = df.sort_values(by="TP_Frequency", ascending=False)
    df['Candidates'] = range(all_hired_ln)
    fig = plt.scatter(df, x='Candidates', y='TP_Frequency', title="TP frequency distribution" ,labels={
                     "TP_Frequency": "TP Frequency"
                 } )
    fig.show()
    
#######Metric1: Coverage (Indicates fairness of the random classifier ensamble)
    all_TP=len(np.nonzero(freq_arr)[0])
    coverage=all_TP/all_hired_ln

    
    
#######Metric2: Arbitrariness (Indicates the arbitrariness of the model and the distribution of true positive)
    TP_freq_mean=np.mean(freq_arr)
    print(TP_freq_mean)
    sum=0
    for j in freq_arr:
        sum=sum+abs(j-TP_freq_mean)

    arbitrariness=(sum/all_hired_ln)

    return coverage, arbitrariness
        

In [1]:
def clean_split_data(data_copy):
    ##############Imputing Null Values############
    data_copy['occupation_skill_1_count']=data_copy['occupation_skill_1_count'].fillna(data_copy['occupation_skill_1_count'].mode()[0])
    data_copy['occupation_skill_2_count']=data_copy['occupation_skill_2_count'].fillna(data_copy['occupation_skill_2_count'].mode()[0])
    data_copy['occupation_skill_3_count']=data_copy['occupation_skill_3_count'].fillna(data_copy['occupation_skill_3_count'].mode()[0])
    data_copy['occupation_skill_4_count']=data_copy['occupation_skill_4_count'].fillna(data_copy['occupation_skill_4_count'].mode()[0])
    data_copy['occupation_skill_5_count']=data_copy['occupation_skill_5_count'].fillna(data_copy['occupation_skill_5_count'].mode()[0])
    data_copy['occupation_skill_6_count']=data_copy['occupation_skill_6_count'].fillna(data_copy['occupation_skill_6_count'].mode()[0])
    data_copy['occupation_skill_7_count']=data_copy['occupation_skill_7_count'].fillna(data_copy['occupation_skill_7_count'].mode()[0])
    data_copy['occupation_skill_8_count']=data_copy['occupation_skill_8_count'].fillna(data_copy['occupation_skill_8_count'].mode()[0])
    data_copy['occupation_skill_9_count']=data_copy['occupation_skill_9_count'].fillna(data_copy['occupation_skill_9_count'].mode()[0])

    data_copy['candidate_attribute_1']=data_copy['candidate_attribute_1'].fillna(data_copy['candidate_attribute_1'].mode()[0])
    data_copy['candidate_attribute_2']=data_copy['candidate_attribute_2'].fillna(round(data_copy['candidate_attribute_2'].mean(),4))
    data_copy['candidate_attribute_3']=data_copy['candidate_attribute_3'].fillna(data_copy['candidate_attribute_3'].mode()[0])
    data_copy['candidate_attribute_4']=data_copy['candidate_attribute_4'].fillna(data_copy['candidate_attribute_4'].mode()[0])
    data_copy['candidate_attribute_5']=data_copy['candidate_attribute_5'].fillna(data_copy['candidate_attribute_5'].mode()[0])
    data_copy['candidate_attribute_6']=data_copy['candidate_attribute_6'].fillna(data_copy['candidate_attribute_6'].mode()[0])
    data_copy['candidate_attribute_7']=data_copy['candidate_attribute_7'].fillna(data_copy['candidate_attribute_7'].mode()[0])
    data_copy['candidate_attribute_8']=data_copy['candidate_attribute_8'].fillna(round(data_copy['candidate_attribute_8'].mean()))

    data_copy['candidate_interest_1']=data_copy['candidate_interest_1'].fillna(data_copy['candidate_interest_1'].mode()[0])
    data_copy['candidate_interest_3']=data_copy['candidate_interest_3'].fillna(data_copy['candidate_interest_3'].mode()[0])
    data_copy['candidate_interest_4']=data_copy['candidate_interest_4'].fillna(data_copy['candidate_interest_4'].mode()[0])
    data_copy['candidate_interest_5']=data_copy['candidate_interest_5'].fillna(data_copy['candidate_interest_5'].mode()[0])
    data_copy['candidate_interest_6']=data_copy['candidate_interest_6'].fillna(data_copy['candidate_interest_6'].mode()[0])
    data_copy['candidate_interest_7']=data_copy['candidate_interest_7'].fillna(data_copy['candidate_interest_7'].mode()[0])
    data_copy['candidate_interest_8']=data_copy['candidate_interest_8'].fillna(data_copy['candidate_interest_8'].mode()[0])

    data_copy['number_years_feature_1']=data_copy['number_years_feature_1'].fillna(round(data_copy['number_years_feature_1'].mean(),6))
    data_copy['number_years_feature_2']=data_copy['number_years_feature_2'].fillna(round(data_copy['number_years_feature_2'].mean(),6))
    data_copy['number_years_feature_3']=data_copy['number_years_feature_3'].fillna(round(data_copy['number_years_feature_3'].mean(),6))
    data_copy['number_years_feature_4']=data_copy['number_years_feature_4'].fillna(round(data_copy['number_years_feature_4'].mean(),6))
    data_copy['number_years_feature_5']=data_copy['number_years_feature_5'].fillna(round(data_copy['number_years_feature_5'].mean(),6))

    data_copy['candidate_skill_1_count']=data_copy['candidate_skill_1_count'].fillna(round(data_copy['candidate_skill_1_count'].mean()))
    data_copy['candidate_skill_2_count']=data_copy['candidate_skill_2_count'].fillna(round(data_copy['candidate_skill_2_count'].mean()))
    data_copy['candidate_skill_3_count']=data_copy['candidate_skill_3_count'].fillna(round(data_copy['candidate_skill_3_count'].mean()))
    data_copy['candidate_skill_4_count']=data_copy['candidate_skill_4_count'].fillna(round(data_copy['candidate_skill_4_count'].mean()))
    data_copy['candidate_skill_5_count']=data_copy['candidate_skill_5_count'].fillna(round(data_copy['candidate_skill_5_count'].mean()))
    data_copy['candidate_skill_6_count']=data_copy['candidate_skill_6_count'].fillna(round(data_copy['candidate_skill_6_count'].mean()))
    data_copy['candidate_skill_7_count']=data_copy['candidate_skill_7_count'].fillna(round(data_copy['candidate_skill_7_count'].mean()))
    data_copy['candidate_skill_8_count']=data_copy['candidate_skill_8_count'].fillna(round(data_copy['candidate_skill_8_count'].mean()))
    data_copy['candidate_skill_9_count']=data_copy['candidate_skill_9_count'].fillna(round(data_copy['candidate_skill_9_count'].mean()))

    data_copy['candidate_relative_test_1']=data_copy['candidate_relative_test_1'].fillna(round(data_copy['candidate_relative_test_1'].mean(),6))
    data_copy['candidate_relative_test_2']=data_copy['candidate_relative_test_2'].fillna(round(data_copy['candidate_relative_test_2'].mean(),6))

    x = data_copy.drop(['application_status', 'candidate_id','occupation_id','company_id','ethnicity','gender','age',\
                        'application_attribute_1', 'candidate_interest_2',\
                        'candidate_demographic_variable_1', 'candidate_demographic_variable_2',\
                        'candidate_demographic_variable_3', 'candidate_demographic_variable_4',\
                        'candidate_demographic_variable_5', 'candidate_demographic_variable_6',\
                        'candidate_demographic_variable_7', 'candidate_demographic_variable_8',\
                        'candidate_demographic_variable_9', 'candidate_demographic_variable_10'],axis=1)

    y = data_copy['application_status']

    #########Splitting data 90 and 10% to test systematicity

    x_train_systemicity, x_test_systemicity, y_train_systemicity, y_test_systemicity = train_test_split(x, y, test_size = 0.1, random_state = 28)

    return x_train_systemicity, x_test_systemicity, y_train_systemicity, y_test_systemicity


In [53]:
#Trainning the ensemble model
def train_ensembleModel(x_train_systemicity, x_test_systemicity, y_train_systemicity, y_test_systemicity, ensenbleModel):

    all_models=[]
    TP_set_list=[]
    FP_set_list=[]
    FN_set_list=[]
    TN_set_list=[]
    all_TP=set()
    all_TP_len=[]
    all_FP=set()
    all_FP_len=[]
    all_FN=set()
    all_FN_len=[]
    all_TN=set()
    all_TN_len=[]
    last_tp_len=0
    last_fp_len=0
    last_fn_len=0
    last_tn_len=0
    total_hired_test_systematicity_set=set(y_test_systemicity.loc[y_test_systemicity=='hired'].index)

    all_TP_len_diff=[]
    last_ten_TP_len_update=[0]*10

    i_tp=0

    flg_strt_running_avg=False

    for i in range(100):

        x_train, x_test, y_train, y_test = train_test_split(x_train_systemicity, y_train_systemicity, test_size = 0.2, random_state = i)

        randomForest = RandomForestClassifier()

        randomForest.set_params(random_state=42,class_weight='balanced',criterion='entropy',min_samples_leaf= 10, min_samples_split= 28, n_estimators= 200)
        randomForest.fit(x_train, y_train) 


        y_pred=randomForest.predict(x_test)



        # using metrics module for accuracy calculation
#         print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

#         print("Confusion matrx OF THE MODEL: ", metrics.confusion_matrix(y_test, y_pred))

#         print("Confusion matrx OF THE MODEL: ", metrics.confusion_matrix(y_test, y_pred).ravel())

        #####TP and FP set

        TP_hired=0
        FN_hired=0
        FP_hired=0
        TN_hired=0
        i=0
        TP_set=set()
        FN_set=set()
        FP_set=set()
        TN_set=set()
        for items in y_test.iteritems():
        #     print(items,"-",items[0],"-",items[1],"---------",y_pred[i])
            if items[1]=="hired":
                if y_pred[i]=="hired":
                    TP_hired=TP_hired+1
                    TP_set.add(items[0])
                else:
                    FN_hired=FN_hired+1
                    FN_set.add(items[0])
            else:
                if y_pred[i]=="hired":
                    FP_hired=FP_hired+1
                    FP_set.add(items[0])
                else:
                    TN_hired=TN_hired+1
                    TN_set.add(items[0])


            i=i+1            

#         print(TP_hired,"-", FN_hired)  
#         print(TP_hired,"-", FN_hired)  
        recall_hired=TP_hired/(TP_hired+FN_hired)
        precision_hired=TP_hired/(TP_hired+FP_hired)
        f1_hired=(2 * precision_hired * recall_hired) / (precision_hired + recall_hired)



        if recall_hired>=0.63 and f1_hired>=0.58:  #Ensuring accuracy


            y_pred_systemicity=randomForest.predict(x_test_systemicity)

            TP_hired_systemicity=0
            FN_hired_systemicity=0
            FP_hired_systemicity=0
            TN_hired_systemicity=0
            i_systemicity=0
            TP_set_systemicity=set()
            FN_set_systemicity=set()
            FP_set_systemicity=set()
            TN_set_systemicity=set()
            for items_systemicity in y_test_systemicity.iteritems():
                if items_systemicity[1]=="hired":
                    if y_pred_systemicity[i_systemicity]=="hired":
                        TP_hired_systemicity=TP_hired_systemicity+1
                        TP_set_systemicity.add(items_systemicity[0])
                    else:
                        FN_hired_systemicity=FN_hired_systemicity+1
                        FN_set_systemicity.add(items_systemicity[0])
                else:
                    if y_pred_systemicity[i_systemicity]=="hired":
                        FP_hired_systemicity=FP_hired_systemicity+1
                        FP_set_systemicity.add(items_systemicity[0])
                    else:
                        TN_hired_systemicity=TN_hired_systemicity+1
                        TN_set_systemicity.add(items_systemicity[0])


                i_systemicity=i_systemicity+1 



            #######################################################################


            if i_tp>=10:
                i_tp=0
                flg_strt_running_avg=True



            if len(TP_set_list)==0:
                all_models.append(randomForest)
                TP_set_list.append(TP_set_systemicity)
                all_TP=all_TP.union(TP_set_systemicity)

                FP_set_list.append(FP_set_systemicity)
                all_FP=all_FP.union(FP_set_systemicity)
                FN_set_list.append(FN_set_systemicity)
                all_FN=all_FN.union(FN_set_systemicity)
                TN_set_list.append(TN_set_systemicity)
                all_TN=all_TN.union(TN_set_systemicity)

                all_TP_len.append(len(all_TP))
                all_FP_len.append(len(all_FP))
                all_FN_len.append(len(all_FN))
                all_TN_len.append(len(all_TN))

                last_tp_len=len(all_TP)


                all_TP_len_diff.append(len(total_hired_test_systematicity_set-all_TP))

            else:
                flg=True
                for itm in TP_set_list:
                    #print("Item : ",itm)
#                     print(len(TP_set_systemicity-itm))
                    if len(TP_set_systemicity-itm)==0:
                        flg=False
                        break
                if flg==True:
                    all_models.append(randomForest)
                    TP_set_list.append(TP_set_systemicity)
                    all_TP=all_TP.union(TP_set_systemicity)

                    FP_set_list.append(FP_set_systemicity)
                    all_FP=all_FP.union(FP_set_systemicity)
                    FN_set_list.append(FN_set_systemicity)
                    all_FN=all_FN.union(FN_set_systemicity)
                    TN_set_list.append(TN_set_systemicity)
                    all_TN=all_TN.union(TN_set_systemicity)

                    all_TP_len.append(len(all_TP))
                    all_FP_len.append(len(all_FP))
                    all_FN_len.append(len(all_FN))
                    all_TN_len.append(len(all_TN))


                    last_ten_TP_len_update[i_tp]=abs(last_tp_len-len(all_TP))
                    last_tp_len=len(all_TP)
                    i_tp=i_tp+1
                    all_TP_len_diff.append(len(total_hired_test_systematicity_set-all_TP))



        if len(total_hired_test_systematicity_set-all_TP)==0:
            break


        if flg_strt_running_avg==True:
            running_avg_diff=sum(last_ten_TP_len_update) / len(last_ten_TP_len_update)
            if running_avg_diff==0:  #if last 10 model has not added atleast one extra new TP
                break



    ########Save model
    mdl_set_len=len(all_models)
    for k in range(mdl_set_len):
        s1="Model"
        s2=".joblib"
        filename = r'{}/{}{}{}'.format(ensenbleModel, s1, k, s2)
    #     print(filename)
        joblib.dump(all_models[k], filename)



In [ ]:
type(all_TP_len)
df_confusion=pd.DataFrame(all_TP_len)

df_confusion.rename( columns={ 0:'TP'}, inplace=True )
df_confusion['Models'] = range(len(df_confusion))
df_confusion
# df.columns = df.columns.str.replace('0','new_name')
count=y_test_systemicity.value_counts().to_dict()
total_hired_test_systematicity=count['hired']

df_confusion['Actual_Positives']=total_hired_test_systematicity

fig = plt.line(df_confusion, x="Models", y="TP", title='L')
fig.show()
df_long=pd.melt(df_confusion, id_vars=['Models'], value_vars=['TP', 'Actual_Positives'])
# fig = plt.line(df_confusion, x="Models", y="TP", title='L')
fig = plt.line(df_long, x='Models', y='value', color='variable')
fig.show()
